In [0]:
dbutils.fs.ls("/FileStore/tables" )

[FileInfo(path='dbfs:/FileStore/tables/FaultDataset-1.csv', name='FaultDataset-1.csv', size=1703184, modificationTime=1682429156000),
 FileInfo(path='dbfs:/FileStore/tables/FaultDataset.csv', name='FaultDataset.csv', size=1703184, modificationTime=1682262899000),
 FileInfo(path='dbfs:/FileStore/tables/Occupancy_Detection_Data.csv', name='Occupancy_Detection_Data.csv', size=50968, modificationTime=1682258711000),
 FileInfo(path='dbfs:/FileStore/tables/Solutions___Recommender_System.zip', name='Solutions___Recommender_System.zip', size=82913, modificationTime=1682326871000),
 FileInfo(path='dbfs:/FileStore/tables/account-models/', name='account-models/', size=0, modificationTime=0),
 FileInfo(path='dbfs:/FileStore/tables/account-models1/', name='account-models1/', size=0, modificationTime=0),
 FileInfo(path='dbfs:/FileStore/tables/accounts/', name='accounts/', size=0, modificationTime=0),
 FileInfo(path='dbfs:/FileStore/tables/activations/', name='activations/', size=0, modificationTime=

In [0]:
#Question 1
clinicaltrial_2021DF = spark.read.csv("/FileStore/tables/clinicaltrial_2021.csv", header= True, inferSchema = True, sep ='|')
distCountDF = clinicaltrial_2021DF.select("Id").distinct().count()
print("Distinct count:", distCountDF)

Distinct count: 387261


In [0]:
#Question 2
clinicaltrial_2021DF.groupBy("Type").count().sort("count",ascending=False).show()

+--------------------+------+
|                Type| count|
+--------------------+------+
|      Interventional|301472|
|       Observational| 77540|
|Observational [Pa...|  8180|
|     Expanded Access|    69|
+--------------------+------+



In [0]:
#Question 3
from pyspark.sql.functions import split
clinicaltrial2021_split=clinicaltrial_2021DF.withColumn('conditions',split(clinicaltrial_2021DF.Conditions,','))
clinicaltrial2021_condition=clinicaltrial2021_split.select(clinicaltrial2021_split.conditions)
from pyspark.sql.functions import explode
clinicaltrial2021_condition_split = clinicaltrial2021_condition.select(explode(clinicaltrial2021_condition.conditions))
clinicaltrial2021_condition_split.groupBy("col").count().sort("count",ascending=False).show(5)

+-----------------+-----+
|              col|count|
+-----------------+-----+
|        Carcinoma|13389|
|Diabetes Mellitus|11080|
|        Neoplasms| 9371|
| Breast Neoplasms| 8640|
|         Syndrome| 8032|
+-----------------+-----+
only showing top 5 rows



In [0]:
#Question 4
pharmaDF = spark.read.csv("/FileStore/tables/pharma.csv", header= True, inferSchema = True)
clinicaltrial_2021_sponsor=clinicaltrial_2021DF.select("Sponsor")
clinicaltrial_2021_parent=pharmaDF.select("Parent_Company")
clinicaltrial_2021_parent_list=clinicaltrial_2021_parent.rdd.map(lambda x: x[0]).collect()
clinicaltrial_2021_pharma_lab =clinicaltrial_2021_sponsor.filter(~clinicaltrial_2021_sponsor.Sponsor.isin(clinicaltrial_2021_parent_list))
clinicaltrial_2021_pharma_lab.groupBy("Sponsor").count().sort("count",ascending=False).show(10)

+--------------------+-----+
|             Sponsor|count|
+--------------------+-----+
|National Cancer I...| 3218|
|M.D. Anderson Can...| 2414|
|Assistance Publiq...| 2369|
|         Mayo Clinic| 2300|
|Merck Sharp & Doh...| 2243|
|   Assiut University| 2154|
|Novartis Pharmace...| 2088|
|Massachusetts Gen...| 1971|
|    Cairo University| 1928|
|   Hoffmann-La Roche| 1828|
+--------------------+-----+
only showing top 10 rows



In [0]:
#Question 5
clinicaltrial_2021_filter=clinicaltrial_2021DF.filter(clinicaltrial_2021DF.Status=='Completed')
clinicaltrial_2021_select=clinicaltrial_2021_filter.select("Completion")
clinicaltrial_2021_month_year = clinicaltrial_2021_select.withColumn('month', split(clinicaltrial_2021_select['Completion'], ' ')[0]) \
       .withColumn('year', split(clinicaltrial_2021_select['Completion'], ' ')[1])
clinicaltrial_2021_year=clinicaltrial_2021_month_year.filter(clinicaltrial_2021_month_year.year=='2021')
clinicaltrial_2021_count_month=clinicaltrial_2021_year.groupBy("month").count().toPandas()
clinicaltrial_2021_list = dict(zip(clinicaltrial_2021_count_month['month'],clinicaltrial_2021_count_month['count']))

months = ['Jan', 'Feb', 'Mar', 'Apr', 'May','Jun', 'Jul', 'Aug', 'Sep', 'Oct', 'Nov', 'Dec']
newList = []
for i in months:
    if i not in clinicaltrial_2021_list:
        break;
    else:
        print(i,":", clinicaltrial_2021_list[i])

Jan : 1131
Feb : 934
Mar : 1227
Apr : 967
May : 984
Jun : 1094
Jul : 819
Aug : 700
Sep : 528
Oct : 187


In [0]:
#My Question 1
pharmaDF_company = pharmaDF.select("Company")
pharmaDF_Offense_Group = pharmaDF.select("Offense_Group")
pharmaDF_parent=clinicaltrial_2021DF.select("Sponsor")

pharmaDF_parent_list=pharmaDF_parent.rdd.map(lambda x: x[0]).collect()

clinicaltrial_2021_pharma_lab =pharmaDF_company.filter(~pharmaDF_company.Company.isin(pharmaDF_parent_list))
clinicaltrial_2021_pharma_lab.groupBy("Company").count().sort("count",ascending=False).show(10)
clinicaltrial_2021_pharma_lab.take(5)


In [0]:
#My Question 2
clinicaltrial_2021_filter1=clinicaltrial_2021DF.filter(clinicaltrial_2021DF.Status=='Recruiting')
clinicaltrial_2021_filter2 = clinicaltrial_2021_filter1.withColumn('month', split(clinicaltrial_2021_filter1['Start'], ' ')[0]) \
       .withColumn('year', split(clinicaltrial_2021_filter1['Start'], ' ')[1])
clinicaltrial_2021_filter3=clinicaltrial_2021_filter2.filter(clinicaltrial_2021_filter2.year==2021)
clinicaltrial_2021_filter3.groupBy("Sponsor").count().sort("count",ascending=False).show(10)


+--------------------+-----+
|             Sponsor|count|
+--------------------+-----+
|Assistance Publiq...|  119|
|         Mayo Clinic|  114|
|M.D. Anderson Can...|   99|
|VA Office of Rese...|   82|
|Novartis Pharmace...|   80|
|Johns Hopkins Uni...|   79|
|     Duke University|   78|
|    Cairo University|   75|
|         AstraZeneca|   73|
|National Cancer I...|   71|
+--------------------+-----+
only showing top 10 rows

